In [1]:
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization


In [11]:
# Load data set
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# CIFAR10 has ten types of images labeled from 0 to 9. We only care about birds, which are labeled as class #2.
# So we'll cheat and change the Y values. Instead of each class being labeled from 0 to 9, we'll set it to True
# if it's a bird and False if it's not a bird.
y_train = (y_train == 2).astype(int)
y_test = (y_test == 2).astype(int)

# Normalize image data (pixel values from 0 to 255) to the 0-to-1 range
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255



In [12]:
# Create a model and add layers
model = Sequential()

# 32 convolution filter & (3,3) kernal & "same" - padding on the image border
model.add(Conv2D(32, (3, 3), padding='same', input_shape=(32, 32, 3), activation="relu"))
model.add(Conv2D(32, (3, 3), activation="relu"))
# 2*2 kernal matrix
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
# remove the 25% of datapoints
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation="relu"))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

In [13]:
# Compile the model
model.compile(
    loss='binary_crossentropy',
    optimizer="adam",
    metrics=['accuracy']
)


In [22]:
# Train the model
model.fit(
    x_train,
    y_train,
    batch_size=16,
    epochs=10,
    validation_data=(x_test, y_test),
    shuffle=True
)

Epoch 1/10
3125/3125 [==============================] - 21s 6ms/step - loss: 0.1238 - accuracy: 0.9537 - val_loss: 0.1962 - val_accuracy: 0.9393
Epoch 2/10
3125/3125 [==============================] - 19s 6ms/step - loss: 0.1103 - accuracy: 0.9589 - val_loss: 0.1717 - val_accuracy: 0.9396
Epoch 3/10
3125/3125 [==============================] - 19s 6ms/step - loss: 0.1089 - accuracy: 0.9585 - val_loss: 0.2193 - val_accuracy: 0.9367
Epoch 4/10
3125/3125 [==============================] - 19s 6ms/step - loss: 0.1101 - accuracy: 0.9595 - val_loss: 0.1863 - val_accuracy: 0.9405
Epoch 5/10
3125/3125 [==============================] - 20s 6ms/step - loss: 0.1058 - accuracy: 0.9604 - val_loss: 0.1776 - val_accuracy: 0.9404
Epoch 6/10
3125/3125 [==============================] - 19s 6ms/step - loss: 0.1003 - accuracy: 0.9627 - val_loss: 0.1731 - val_accuracy: 0.9391
Epoch 7/10
3125/3125 [==============================] - 20s 6ms/step - loss: 0.1012 - accuracy: 0.9616 - val_loss: 0.2864 - val_ac

In [23]:
# Save the trained model to a file so we can use it to make predictions later
model.save("bird_model2.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [24]:
from keras.models import load_model
from sklearn.metrics import confusion_matrix, classification_report

model = load_model('bird_model2.h5')
predictions = model.predict(x_test, batch_size=32, verbose=1)

# If the model is more than 50% sure the object is a bird, call it a bird.
# Otherwise, call it "not a bird".
predictions = predictions > 0.5

# Calculate how many mis-classifications the model makes
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
print("True Positives: ",tp)
print("True Negatives: ",tn)
print("False Positives: ",fp)
print("False Negatives: ",fn)

# Calculate Precision and Recall for each class
report = classification_report(y_test, predictions)
print(report)

313/313 [==============================] - 1s 2ms/step
True Positives:  553
True Negatives:  8881
False Positives:  119
False Negatives:  447
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      9000
           1       0.82      0.55      0.66      1000

    accuracy                           0.94     10000
   macro avg       0.89      0.77      0.82     10000
weighted avg       0.94      0.94      0.94     10000



In [28]:
from keras.models import load_model
from keras.preprocessing import image
from pathlib import Path
import numpy as np

# Load the model we trained
model = load_model('bird_model2.h5')

for f in sorted(Path(".").glob("*.png")):

    # Load an image file to test
    image_to_test = image.load_img(str(f), target_size=(32, 32))

    # Convert the image data to a numpy array suitable for Keras
    image_to_test = image.img_to_array(image_to_test)

    # Normalize the image the same way we normalized the training data (divide all numbers by 255)
    image_to_test /= 255

    # Add a fourth dimension to the image since Keras expects a list of images
    list_of_images = np.expand_dims(image_to_test, axis=0)

    # Make a prediction using the bird model
    results = model.predict(list_of_images)

    # Since we only passed in one test image, we can just check the first result directly.
    image_likelihood = results[0][0]

    # The result will be a number from 0.0 to 1.0 representing the likelihood that this image is a bird.
    if image_likelihood > 0.5:
        print(f"{f} is most likely a bird! ({image_likelihood:.2f})")
    else:
        print(f"{f} is most likely NOT a bird! ({image_likelihood:.2f})")

1/1 [==============================] - 0s 93ms/step
bird1.png is most likely a bird! (0.97)
1/1 [==============================] - 0s 18ms/step
bird2.png is most likely a bird! (0.85)
1/1 [==============================] - 0s 19ms/step
bird3.png is most likely a bird! (1.00)
1/1 [==============================] - 0s 18ms/step
bird4.png is most likely a bird! (0.64)
1/1 [==============================] - 0s 21ms/step
bird5.png is most likely a bird! (1.00)
1/1 [==============================] - 0s 18ms/step
not_bird1.png is most likely NOT a bird! (0.00)
1/1 [==============================] - 0s 17ms/step
not_bird2.png is most likely NOT a bird! (0.00)
1/1 [==============================] - 0s 26ms/step
not_bird3.png is most likely NOT a bird! (0.03)
1/1 [==============================] - 0s 18ms/step
not_bird4.png is most likely NOT a bird! (0.00)
1/1 [==============================] - 0s 21ms/step
not_bird5.png is most likely NOT a bird! (0.00)
